# Livedoorニュースコーパスを用いたニューストピック分類

### タスクの説明
LiveDoorニュースコーパスを用いて、ニュース記事のトピック分類を行います。  
本資料ではすでに加工済みのデータを利用しています。  
元データのダウンドードは[こちらから](https://www.rondhuit.com/download.html)行えます。  

このコーパスは各記事がどのトピックに属するかがラベル付けされています。
このコーパスに含まれるトピックは以下の通りです。
- トピックニュース
- Sports Watch
- ITライフハック
- 家電チャンネル
- MOVIE ENTER
- 独女通信
- エスマックス
- livedoor HOMME
- Peachy

今回実装するタスクは、ニュース記事の本文から上記の九つのトピックのうちどれに属するかを予測するマルチクラス分類です。

In [1]:
# GPUが複数台搭載されている実行環境の場合、以下の環境変数を設定することで、使用するGPUを指定できます
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
# 必要なライブラリのインストール
# NOTE: 仮想環境が適切に構築されていることを事前に確認して下さい
!pip install pandas transformers torch fugashi ipadic

### モデルの定義

In [4]:
# seed値の設定
from transformers import set_seed
seed = 42
set_seed(seed)

/home/mbaba/dev/python/dpp2024/part3/practice/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=num_labels)
print(model.classifier)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=9, bias=True)


### Livedoorニュースコーパスの読み込み

In [6]:
with open("./content/categories.json","r") as f:
    d = f.read()
    import json
    categories = json.loads(d)
num_labels = len(categories)
print(categories)
print(f"num_labels: {num_labels}")

[{'label': 0, 'category': 'movie-enter'}, {'label': 1, 'category': 'peachy'}, {'label': 2, 'category': 'livedoor-homme'}, {'label': 3, 'category': 'kaden-channel'}, {'label': 4, 'category': 'dokujo-tsushin'}, {'label': 5, 'category': 'sports-watch'}, {'label': 6, 'category': 'smax'}, {'label': 7, 'category': 'it-life-hack'}, {'label': 8, 'category': 'topic-news'}]
num_labels: 9


### データセットの準備

In [28]:
import pandas as pd
df_train = pd.read_csv("./content/train.csv")
df_valid = pd.read_csv("./content/valid.csv")
df_test = pd.read_csv("./content/test.csv")



creating train dataset...(this may take a while)
creating valid dataset...
creating test dataset...


In [ ]:
print(df_train)

In [8]:
# tokenizerの読み込み
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# モデルの最大入力トークン数
max_length = tokenizer.model_max_length
print(max_length)

512


In [27]:
# 今回用意されたcsvの形式が読み込まれたdata frameからtokenize後のデータセットを作成する関数を定義
import random
def create_dataset(df):
    dataset = []
    for i in range(len(df)):
        label = df.iloc[i, 0]
        text = df.iloc[i, 4]
        if label == "9" & random.randint(1, 10) > 1:
            continue
        dataset.append({
            "label": label,
            "text": text,
            "encoding": tokenizer(text, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
        })
    return dataset

In [ ]:
# 読み込んだデータフレームから１レコードが辞書型として格納されている配列を作成
print("creating train dataset...(this may take a while)")
train_dataset = create_dataset(df_train)
print("creating valid dataset...")
valid_dataset = create_dataset(df_valid)
print("creating test dataset...")
test_dataset = create_dataset(df_test)

In [21]:
# データセットを確認
print(f"Number of samples: {len(train_dataset)}")
print(f"Sample 0: {train_dataset[0]}")

Number of samples: 5893
Sample 0: {'text': '動物好きなアナタに！ iPhoneを動物図鑑にしてくれるアプリ【iPhoneでチャンスを掴め】子供の頃、野生の動物を紹介するテレビ番組にかじりついていたなんて人いらっしゃいませんか？\u3000眺めているだけでも楽しい動物図鑑のアプリを紹介します。こんな人におすすめ：動物について英語で学びたい人、動物の写真を見たい人こんなときにおすすめ：動物について知りたいとき、動物の知識が欲しいときアプリ名：Animal Kingdom for iPhoneカテゴリ：教育価格：無料「子どもの頃、動物図鑑を飽きずにずっと眺めていた」「野生の動物を紹介するテレビは欠かさず見ていた」という方はいらっしゃいませんか？\u3000そんな方におすすめのアプリです。動物図鑑がiPhoneアプリになりました。すべて英語ですが、眺めているだけでも十分に楽しめます。お子様でも楽しめるアプリです。ほら！動物の画像がこんなにたくさん！\u3000好きな画像を選びましょう。すると動物の説明が！\u3000英語ですが「サイズ」、「寿命」、「ライフスタイル」など、分かりやすくなっていますので英語の勉強にもなりますね。図鑑を一通り眺め終わったら、クイズに挑戦してみましょう！もちろん、名前で検索もできます。動物のスペルを知っていますか？\u3000さらに動物の画像は、加工して保存したり、あるいは直接メールに添付できます。\u3000動物好きな人にはたまらないアプリですね。■RainbowApps - iPhoneアプリ開発を学ぶなら1000人が受講したEagleが運営する東京校が一番！■成長率 888% 毎日 50万台増える Androidアプリに参入しよう。Androidマスターコース■人気のAngryBirds風アプリやスーパーマリオ風アプリが簡単に作れちゃう。ゲームアプリコース■iPhoneの日米人気アプリランキングをチェックするからRainbowApps■RainbowAppsの記事をもっと読む', 'label': 7, 'input_ids': {'input_ids': tensor([[    2,  2056,  3596,    18,  1742, 28502,     7,   679, 18060, 

In [29]:
# Dataloaderの作成
from torch.utils.data import DataLoader

batch_size = 64  # Hyperparameter

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(valid_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")


Number of training samples: 5893
Number of validation samples: 736
Number of test samples: 738


In [30]:
import torch

lr = 1e-5 # Hyperparameter
optimizer_parameters = model.parameters()
optimizer = torch.optim.Adam(optimizer_parameters, lr=lr)

In [31]:
# 学習を行うデバイスを設定
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(f"device: {device}")

cuda


In [32]:
from tqdm import tqdm

losses = []

epoch_num = 3 # Hyperparameter

step_num = 1
epochs = 1

last_loss = float('inf')
loss_threshold = 0.02 # Hyperparameter

valid_step_period = len(train_dataloader) // 5

model.to(device)

print("train start!!!")
for _ in range(epoch_num):
    print(f'########### Epoch {epochs} ###########')
    # train step
    for data in train_dataloader:
        optimizer.zero_grad()
        input_ids = data["encoding"]["input_ids"].squeeze(1).to(device)
        attention_mask = data["encoding"]["attention_mask"].squeeze(1).to(device)
        token_type_ids = data["encoding"]["token_type_ids"].squeeze(1).to(device)
        labels = data["label"].to(device)
    
        model_output = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
        logit = model_output.logits
        loss = model_output.loss
        output_label = torch.argmax(logit, dim=1)
    
        losses.append(loss.item())
    
        loss.backward()
        optimizer.step()
        last_loss = loss.item()
        step_num += 1
    
        # validation step
        if step_num % valid_step_period == 0:
            valid_output_labels = []
            valid_correct_labels = []
            for data in valid_dataloader:
                labels = data["label"].to(device)
                
                model_output = model(**{k: v.squeeze(1).to(device) for k, v in data["encoding"].items()}, labels=labels)
                # モデルの最終層の出力
                logit = model_output.logits
                # モデルの最終層の出力が最も高いラベルを予測ラベルとする
                output_label = torch.argmax(logit, dim=1)

                valid_output_labels.append(output_label)
                valid_correct_labels.append(labels)

            # accuracyの算出：モデルの出力ラベルと正解ラベルが一致している数を全データ数で割った値を算出
            valid_accuracy = torch.sum(torch.cat(valid_output_labels) == torch.cat(valid_correct_labels)).item() / len(valid_dataset)
            print(f"Step {step_num} / {len(train_dataloader)}, Last Loss {last_loss}, Valid Accuracy {valid_accuracy}")
            # モデルを学習モードに戻す
            model.train()

        # early stopping
        if last_loss < loss_threshold:
            print("Loss is less than threshold!")
            break
    epochs += 1
    step = 0
print("train finish!!!")

train start!!!
########### 1 Epochs ###########
Step 18 / 92, Last Loss 2.0047366619110107, Valid Accuracy 0.45108695652173914
Step 36 / 92, Last Loss 1.5418059825897217, Valid Accuracy 0.625
Step 54 / 92, Last Loss 1.1709026098251343, Valid Accuracy 0.7567934782608695
Step 72 / 92, Last Loss 0.957701563835144, Valid Accuracy 0.7717391304347826
Step 90 / 92, Last Loss 0.8178668022155762, Valid Accuracy 0.8233695652173914
########### 2 Epochs ###########
Step 108 / 92, Last Loss 0.6956350803375244, Valid Accuracy 0.8586956521739131
Step 126 / 92, Last Loss 0.4833647906780243, Valid Accuracy 0.8967391304347826
Step 144 / 92, Last Loss 0.35873493552207947, Valid Accuracy 0.8953804347826086
Step 162 / 92, Last Loss 0.2874962091445923, Valid Accuracy 0.9198369565217391
Step 180 / 92, Last Loss 0.37805742025375366, Valid Accuracy 0.9089673913043478
########### 3 Epochs ###########
Step 198 / 92, Last Loss 0.2096950262784958, Valid Accuracy 0.9429347826086957
Step 216 / 92, Last Loss 0.316420

In [33]:
# test step
test_correct_num = 0
test_total_num = len(test_dataset)

model.eval()
for data in tqdm(test_dataloader):
    labels = data["label"].to(device)

    model_output = model(**{k: v.squeeze().to(device) for k,v in data["encoding"].items()}, labels=labels)
    logit = model_output.logits
    output_label = torch.argmax(logit, dim=1)

    test_correct_num += torch.sum(output_label == labels).item()

100%|██████████| 738/738 [00:06<00:00, 109.34it/s]


In [35]:
accuracy = test_correct_num / test_total_num
print(f"correct samples | total samples : {test_correct_num} | {test_total_num}")
print(f"Accuracy: {accuracy}")

correct samples | total samples : 703 | 738
Accuracy: 0.9525745257452575


In [43]:
# モデルの保存
model_dir = "./content/model/"
model_full_path = model_dir + "model.pth"
os.makedirs(model_dir, exist_ok=True)
torch.save(model.state_dict(), model_full_path)